# Kütüphaneler

In [2]:
import sys
import joblib
import pandas as pd
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QComboBox, QSpinBox, QPushButton,
    QVBoxLayout, QHBoxLayout
)
from PyQt5.QtGui import QPixmap, QPalette, QBrush, QPainter, QColor, QFont, QIcon
from PyQt5.QtCore import Qt
from sklearn.preprocessing import LabelEncoder


# KoltukGorselPenceresi sınıfı

In [5]:
class KoltukGorselPenceresi(QWidget):
    def __init__(self, en_iyi_koltuklar):
        super().__init__()
        self.setWindowTitle("Koltuk Haritası")
        self.setWindowIcon(QIcon("app_icon.png"))
        self.setFixedSize(720, 480)
        self.en_iyi = en_iyi_koltuklar
        self.setStyleSheet("background-color: white;")
        self.show()

    def paintEvent(self, event):
        painter = QPainter(self)
        arka_plan = QPixmap("background2.png")
        painter.drawPixmap(self.rect(), arka_plan)

        rows = list("ABCDEFGHIJK")
        cols = list(range(1, 13))
        padding = 30
        cell_w = (self.width() - 2 * padding) / 12
        cell_h = (self.height() - 2 * padding) / 11

        font = QFont("Segoe UI", 8)
        painter.setFont(font)

        for i, row in enumerate(rows):
            for j, col in enumerate(cols):
                x = padding + j * cell_w
                y = padding + i * cell_h

                label = f"{row}{col}"
                if label in self.en_iyi:
                    painter.setBrush(QColor(0, 200, 0))  # Yeşil
                else:
                    painter.setBrush(QColor(230, 230, 230))  # Gri
                painter.setPen(Qt.black)
                painter.drawRect(int(x), int(y), int(cell_w - 5), int(cell_h - 5))
                painter.drawText(int(x + 5), int(y + 15), label)

        painter.end()


# KoltukOneriApp sınıfı

In [8]:
class KoltukOneriApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Tiyatro Koltuk Seçimi")
        self.setWindowIcon(QIcon("app_icon.png"))
        self.setFixedSize(960, 540)

        self.model_tercih = joblib.load("tercih_modeli.pkl")
        self.model_memnuniyet = joblib.load("memnuniyet_modeli.pkl")
        self.df_orig = pd.read_csv("koltuk_verisi.csv")
        self.feature_order = list(self.model_tercih.feature_names_in_)
        self.label_encoders = {}
        self.fit_encoders()

        self.setAutoFillBackground(True)
        palette = QPalette()
        pixmap = QPixmap("background1.png")
        palette.setBrush(QPalette.Window, QBrush(pixmap))
        self.setPalette(palette)

        self.input_style = """
            QComboBox, QSpinBox {
                border: 1px solid #d1d5db;
                border-radius: 10px;
                padding: 6px 12px;
                font-size: 11pt;
                font-family: 'Segoe UI';
                background-color: white;
            }
            QComboBox::drop-down, QSpinBox::up-button, QSpinBox::down-button {
                width: 0;
                height: 0;
                border: none;
            }
            QComboBox::down-arrow, QSpinBox::up-arrow, QSpinBox::down-arrow {
                width: 0;
                height: 0;
            }
        """
        self.init_ui()

    def fit_encoders(self):
        df = self.df_orig.copy()
        for col in ["Cinsiyet", "Sahne Yakınlığı Tercihi", "Fiyat Kategorisi", "Sıra"]:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            self.label_encoders[col] = le

    def init_ui(self):
        self.form_area = QWidget(self)
        self.form_area.setStyleSheet("""
            background-color: rgba(255, 255, 255, 230);
            border-radius: 15px;
            padding: 20px;
        """)
        self.form_area.setGeometry(230, 130, 500, 400)

        form_layout = QVBoxLayout()

        def form_row(label, widget):
            row = QHBoxLayout()
            lbl = QLabel(label)
            lbl.setStyleSheet("""
                padding: 5px 10px;
                font-size: 11pt;
                font-family: 'Segoe UI';
                color: #1f2937;
                min-width: 120px;
            """)
            widget.setStyleSheet(self.input_style)
            row.addWidget(lbl)
            row.addWidget(widget)
            return row

        self.age_box = QSpinBox(); self.age_box.setRange(10, 100)
        self.experience_box = QSpinBox(); self.experience_box.setRange(0, 10)
        self.sound_quality_box = QSpinBox(); self.sound_quality_box.setRange(1, 10)

        self.gender_box = QComboBox(); self.gender_box.addItems(self.label_encoders["Cinsiyet"].classes_)
        self.stage_pref_box = QComboBox(); self.stage_pref_box.addItems(self.label_encoders["Sahne Yakınlığı Tercihi"].classes_)
        self.price_box = QComboBox(); self.price_box.addItems(self.label_encoders["Fiyat Kategorisi"].classes_)

        for box in [self.gender_box, self.stage_pref_box, self.price_box]:
            box.setEditable(False)
            box.view().setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOff)

        form_layout.addLayout(form_row("Yaş:", self.age_box))
        form_layout.addLayout(form_row("Deneyim:", self.experience_box))
        form_layout.addLayout(form_row("Ses Kalitesi:", self.sound_quality_box))
        form_layout.addLayout(form_row("Cinsiyet:", self.gender_box))
        form_layout.addLayout(form_row("Sahne Tercihi:", self.stage_pref_box))
        form_layout.addLayout(form_row("Fiyat Tercihi:", self.price_box))

        self.output = QLabel("🎟️ En İyi 5 Koltuk: ")
        self.output.setFixedHeight(40)
        self.output.setAlignment(Qt.AlignCenter)
        self.output.setStyleSheet("""
            background-color: white;
            border: 1px solid #ccc;
            border-radius: 10px;
            padding: 8px;
            font-size: 11pt;
            font-family: 'Segoe UI';
        """)
        form_layout.addWidget(self.output)

        button = QPushButton("🎯  En İyi Koltukları Göster")
        button.setStyleSheet("""
            background-color: #3b82f6;
            color: white;
            font-size: 11pt;
            font-weight: 600;
            border-radius: 10px;
            padding: 8px;
            letter-spacing: 0px;
            border: none;
        """)
        button.clicked.connect(self.run_model)
        form_layout.addWidget(button)

        self.form_area.setLayout(form_layout)

    def run_model(self):
        user_static = {
            "Yaş": self.age_box.value(),
            "Cinsiyet": self.label_encoders["Cinsiyet"].transform([self.gender_box.currentText()])[0],
            "Tiyatro Deneyimi": self.experience_box.value(),
            "Sahne Yakınlığı Tercihi": self.label_encoders["Sahne Yakınlığı Tercihi"].transform([self.stage_pref_box.currentText()])[0],
            "Ses Kalitesi": self.sound_quality_box.value(),
            "Fiyat Kategorisi": self.label_encoders["Fiyat Kategorisi"].transform([self.price_box.currentText()])[0],
        }

        base_cols = ["Sıra", "Koltuk"] + [col for col in self.df_orig.columns if col in self.feature_order and col not in user_static]
        base_cols = list(dict.fromkeys(base_cols))
        aday = self.df_orig[base_cols].drop_duplicates(subset=["Sıra", "Koltuk"]).copy()
        aday["Sıra"] = self.label_encoders["Sıra"].transform(aday["Sıra"])

        for k, v in user_static.items():
            aday[k] = v
        for col in self.feature_order:
            if col not in aday.columns:
                aday[col] = 0

        X = aday[self.feature_order]
        aday["Tercih"] = self.model_tercih.predict(X)
        secim = aday[aday["Tercih"] == 1].copy()

        if secim.empty:
            self.output.setText("❌ Uygun koltuk bulunamadı.")
            return

        secim["Memnuniyet Tahmini"] = self.model_memnuniyet.predict(secim[self.feature_order])
        en_iyi = secim.sort_values("Memnuniyet Tahmini", ascending=False).head(5)
        en_iyi["Sıra"] = self.label_encoders["Sıra"].inverse_transform(en_iyi["Sıra"])

        koltuklar = [f"{row['Sıra']}{int(row['Koltuk'])}" for _, row in en_iyi.iterrows()]
        self.output.setText("🎟️ En İyi 5 Koltuk: " + ", ".join(koltuklar))

        self.gorsel_pencere = KoltukGorselPenceresi(koltuklar)


# GUI’yi Başlat

In [11]:
app = QApplication(sys.argv)
window = KoltukOneriApp()
window.show()
sys.exit(app.exec_())

SystemExit: 0

C:\Users\Sefa\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
